# Data Exploration: Elliptic Dataset
# Augmentate network features
https://www.kaggle.com/datasets/ellipticco/elliptic-data-set/data

[1] Elliptic, www.elliptic.co.

[2] M. Weber, G. Domeniconi, J. Chen, D. K. I. Weidele, C. Bellei, T. Robinson, C. E. Leiserson, "Anti-Money Laundering in Bitcoin: Experimenting with Graph Convolutional Networks for Financial Forensics", KDD ’19 Workshop on Anomaly Detection in Finance, August 2019, Anchorage, AK, USA.

Description: The Elliptic Data Set maps Bitcoin transactions to real entities belonging to licit categories (exchanges, wallet providers, miners, licit services, etc.) versus illicit ones (scams, malware, terrorist organizations, ransomware, Ponzi schemes, etc.). The task on the dataset is to classify the illicit and licit nodes in the graph.


In [50]:
import networkx as nx
import pandas as pd
from pathlib import Path
import plotly
import numpy as np
import plotly.graph_objects as go

from src.data.load_data import load_elliptic_dataset
from src.visualization.graphs import plot_transaction_graph
from src.data.preprocess import corr_with_binary_labels
from scipy import stats
from scipy.stats import chi2_contingency
pd.options.plotting.backend = 'plotly'
from typing import List, Dict
from src.features.network_features import *

In [12]:
import os
import pathlib
import random
from typing import Dict, List
from tqdm import tqdm

import numpy as np
import pandas as pd
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv

In [5]:
DATA_PATH = 'data/raw/'
nodes_df, edges_df = load_elliptic_dataset(DATA_PATH)
nodes_df['class_label'] = nodes_df['class_label'].replace(['1', '2', 'unknown'], ['illicit', 'licit', 'unknown'])

In [14]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [47]:
G = build_nx_graph(edges_df)
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
labels = create_label_dict(nodes_df)
print("Building time step graphs...")
time_step_graphs = build_time_step_graphs(edges_df, nodes_df)
print(f"Created {len(time_step_graphs)} separate graphs for different time steps")

Graph has 203769 nodes and 234355 edges
Building time step graphs...
Created 49 separate graphs for different time steps


In [49]:
# 4. Compute GuiltyWalker features
print("Computing GuiltyWalker features...")
gw_features = guilty_walker_features(nodes_df, time_step_graphs)

Computing GuiltyWalker features...


Processing time steps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [07:01<00:00,  8.60s/it]


In [53]:
gw_features.merge(nodes_df[['txId', 'class_label']], on = 'txId').query('class_label == "illicit" & gw_avg_step != 1')

,txId,gw_hit_ratio,gw_min_step,gw_avg_step,gw_unique_illicit,class_label
907,232629023,0.0,51,51.0,0,illicit
1361,230389796,0.0,51,51.0,0,illicit
2718,17387772,0.0,51,51.0,0,illicit
2815,232947878,0.0,51,51.0,0,illicit
3423,16754007,0.0,51,51.0,0,illicit
...,...,...,...,...,...,...
203685,159043651,0.0,51,51.0,0,illicit
203708,158360779,0.0,51,51.0,0,illicit
203736,159028476,0.0,51,51.0,0,illicit
203759,158375075,0.0,51,51.0,0,illicit


In [64]:
mps_device = torch.device("mps")
feature_cols = nodes_df.drop(columns=['txId', 'time_step', 'class_label']).columns.to_list()
# 5. Train GCN and get embeddings
gcn_embeddings = train_gcn_get_embeddings(nodes_df, time_step_graphs, feature_cols, device=mps_device, epochs =400)
    

GCN Embeddings:   0%|                                                                                                                                              | 0/49 [00:00<?, ?it/s]/Users/nick/AML/src/features/network_features.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(train_mask, dtype=torch.bool)


Time step 1, Epoch 0: Loss 9.2096
Time step 1, Epoch 10: Loss 1.3408
Time step 1, Epoch 20: Loss 0.3778
Time step 1, Epoch 30: Loss 0.1554
Time step 1, Epoch 40: Loss 0.0971
Time step 1, Epoch 50: Loss 0.0733
Time step 1, Epoch 60: Loss 0.0591
Time step 1, Epoch 70: Loss 0.0531
Time step 1, Epoch 80: Loss 0.0442
Time step 1, Epoch 90: Loss 0.0450
Time step 1, Epoch 100: Loss 0.0406
Time step 1, Epoch 110: Loss 0.0388
Time step 1, Epoch 120: Loss 0.0367
Time step 1, Epoch 130: Loss 0.0313
Time step 1, Epoch 140: Loss 0.0301
Time step 1, Epoch 150: Loss 0.0279
Time step 1, Epoch 160: Loss 0.0310
Time step 1, Epoch 170: Loss 0.0255
Time step 1, Epoch 180: Loss 0.0265
Time step 1, Epoch 190: Loss 0.0228
Time step 1, Epoch 200: Loss 0.0230
Time step 1, Epoch 210: Loss 0.0245
Time step 1, Epoch 220: Loss 0.0232
Time step 1, Epoch 230: Loss 0.0191
Time step 1, Epoch 240: Loss 0.0222
Time step 1, Epoch 250: Loss 0.0188
Time step 1, Epoch 260: Loss 0.0192
Time step 1, Epoch 270: Loss 0.0173
Tim

GCN Embeddings:   2%|██▋                                                                                                                                   | 1/49 [00:06<05:20,  6.67s/it]/Users/nick/AML/src/features/network_features.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(train_mask, dtype=torch.bool)


Time step 2, Epoch 0: Loss 5.7882
Time step 2, Epoch 10: Loss 1.3592
Time step 2, Epoch 20: Loss 0.3977
Time step 2, Epoch 30: Loss 0.1860
Time step 2, Epoch 40: Loss 0.1350
Time step 2, Epoch 50: Loss 0.0960
Time step 2, Epoch 60: Loss 0.0768
Time step 2, Epoch 70: Loss 0.0618
Time step 2, Epoch 80: Loss 0.0558
Time step 2, Epoch 90: Loss 0.0489
Time step 2, Epoch 100: Loss 0.0428
Time step 2, Epoch 110: Loss 0.0395
Time step 2, Epoch 120: Loss 0.0371
Time step 2, Epoch 130: Loss 0.0317
Time step 2, Epoch 140: Loss 0.0307
Time step 2, Epoch 150: Loss 0.0291
Time step 2, Epoch 160: Loss 0.0268
Time step 2, Epoch 170: Loss 0.0241
Time step 2, Epoch 180: Loss 0.0250
Time step 2, Epoch 190: Loss 0.0220
Time step 2, Epoch 200: Loss 0.0205
Time step 2, Epoch 210: Loss 0.0210
Time step 2, Epoch 220: Loss 0.0181
Time step 2, Epoch 230: Loss 0.0168
Time step 2, Epoch 240: Loss 0.0146
Time step 2, Epoch 250: Loss 0.0194
Time step 2, Epoch 260: Loss 0.0143
Time step 2, Epoch 270: Loss 0.0151
Tim

GCN Embeddings:   4%|█████▍                                                                                                                                | 2/49 [00:10<03:57,  5.06s/it]

...

Time step 48, Epoch 390: Loss 0.0149


/Users/nick/AML/src/features/network_features.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(train_mask, dtype=torch.bool)


Time step 49, Epoch 0: Loss 4.0436
Time step 49, Epoch 10: Loss 1.1105
Time step 49, Epoch 20: Loss 0.4244
Time step 49, Epoch 30: Loss 0.2112
Time step 49, Epoch 40: Loss 0.1525
Time step 49, Epoch 50: Loss 0.1291
Time step 49, Epoch 60: Loss 0.0960
Time step 49, Epoch 70: Loss 0.0824
Time step 49, Epoch 80: Loss 0.0705
Time step 49, Epoch 90: Loss 0.0673
Time step 49, Epoch 100: Loss 0.0692
Time step 49, Epoch 110: Loss 0.0558
Time step 49, Epoch 120: Loss 0.0482
Time step 49, Epoch 130: Loss 0.0442
Time step 49, Epoch 140: Loss 0.0409
Time step 49, Epoch 150: Loss 0.0340
Time step 49, Epoch 160: Loss 0.0373
Time step 49, Epoch 170: Loss 0.0392
Time step 49, Epoch 180: Loss 0.0356
Time step 49, Epoch 190: Loss 0.0292
Time step 49, Epoch 200: Loss 0.0252
Time step 49, Epoch 210: Loss 0.0272
Time step 49, Epoch 220: Loss 0.0288
Time step 49, Epoch 230: Loss 0.0241
Time step 49, Epoch 240: Loss 0.0209
Time step 49, Epoch 250: Loss 0.0172
Time step 49, Epoch 260: Loss 0.0188
Time step 49

GCN Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [03:26<00:00,  4.22s/it]

Time step 49, Epoch 390: Loss 0.0081


In [66]:
# 5. Merge all features
print("Merging features...")
final_df = nodes_df.merge(gw_features, on='txId', how='left')
final_df = final_df.merge(gcn_embeddings, on='txId', how='left')
final_df.to_parquet('data/processed/df_model.parquet', partition_cols='time_step')

Merging features...


In [ ]:

    # 4. Compute GuiltyWalker features
    print("Computing GuiltyWalker features...")
    gw_features = guilty_walker_features(G, labels)
    
    # 5. Train GCN and get embeddings
    print("Training GCN and getting embeddings...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    gcn_embeddings = train_gcn_get_embeddings(G, df, feature_cols, device=device)
    
    # 6. Merge all features
    print("Merging features...")
    final_df = df.merge(gw_features, on='txId', how='left')
    final_df = final_df.merge(gcn_embeddings, on='txId', how='left')
    
    # 7. Train classifier (time-based split as in the paper)
    print("Training classifier...")
    # Split by timestep (34 is the cutoff mentioned in the paper)
    train_mask = final_df['time_step'] <= 34
    test_mask = final_df['time_step'] > 34
    
    # Prepare features
    X_cols = feature_cols + [c for c in final_df.columns if c.startswith('gw_') or c.startswith('gcn_')]
    X = final_df[X_cols].values
    # Class '2' is illicit
    y = (final_df['class_label'] == '2').astype(int).values
    
    X_train = X[train_mask]
    y_train = y[train_mask]
    X_test = X[test_mask]
    y_test = y[test_mask]
    
    # Train Random Forest
    clf = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42)
    clf.fit(X_train, y_train)
    
    # Evaluate
    y_pred = clf.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    